In [1]:
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import re
import time
import unidecode

In [2]:
Base = declarative_base()

In [3]:
class Artist(Base):
    __tablename__ = 'artists'
    id = Column(Integer, primary_key=True)
    name = Column(Text)
    gender = Column(Text)
    birthday = Column(Integer)
    deathday = Column(Integer)
#     hometown = Column(Text)
    location = Column(Text)
    nationality = Column(Text)
    years = relationship('Year', back_populates='artist')
    genes = relationship('Gene', secondary='artists_genes')

In [4]:
class Year(Base):
    __tablename__ = 'years'
    id = Column(Integer, primary_key=True)
    year = Column(Integer)
    rank = Column(Integer)
    artist_id = Column(Integer, ForeignKey('artists.id'))
    artist = relationship('Artist', back_populates='years')
    totalsold = Column(Integer)
    totallots =  Column(Integer)
    maxprice = Column(Integer)

In [5]:
#if no display_name, name
class Gene(Base):
    __tablename__= 'genes'
    id = Column(Integer, primary_key=True)
    name = Column(Text)
    artists = relationship('Artist', secondary='artists_genes')

In [6]:
class ArtistGene(Base):
    __tablename__ = 'artists_genes'
    artist_id = Column(Integer, ForeignKey('artists.id'), primary_key=True)
    gene_id = Column(Integer, ForeignKey('genes.id'), primary_key=True)

In [7]:
engine = create_engine('sqlite:///artsy.db')
Base.metadata.create_all(engine)

In [8]:
Session = sessionmaker(bind=engine)
session = Session()

In [9]:
from artists2010 import clean2010top500

In [10]:
"-".join(clean2010top500[400]['Name']).lower()

'niki-de-saint-phalle'

In [11]:
import requests
import json

client_id = '5a2c7ea90a393fa55c65'
client_secret = '5cda60eedbc43c586621c30d6d19210e'

r = requests.post('https://api.artsy.net/api/tokens/xapp_token',
                      data={'client_id': client_id, 'client_secret': client_secret})

token_text = json.loads(r.text)

token = token_text['token']

headers = {'X-Xapp-Token': token}

list_answer = {}
artists = 'qi-baishi'

# api_url='https://api.artsy.net/api/artists/'+artists.lower()
api_url = 'https://api.artsy.net/api/search?offset=0&q=ZHU+DA'
res = requests.get(api_url, headers=headers)
res.encoding = 'utf-8'

In [12]:
res.json()['_embedded']['results'][0]['_links']['self']['href']

'https://api.artsy.net/api/artists/4ee262199de8920001000530'

In [44]:
def verify_artist(json_thing):
    if json_thing.get('type') != 'other_error':
        return True
    elif json_thing.get('type') == 'other_error':
        return False

# Order is fine, gotta figure 404s
def query_json(artist_name):
    artist_name = [re.sub("[')(.]", '', x)  for x in artist_name]
    artist_name = [unidecode.unidecode(y) for y in artist_name]
    name = "-".join(artist_name)
    api_url='https://api.artsy.net/api/artists/'+name.lower()
    res = requests.get(api_url, headers=headers)
    res.encoding = 'utf-8'
    verifiable = verify_artist(res.json())
    if  verifiable == True:
        #debugging
        print('verified true')
        print(res)
        #########
        return res.json()
    elif verifiable == False :
        new_name = list(reversed(artist_name))
        name = "-".join(new_name)
        api_url='https://api.artsy.net/api/artists/'+name.lower()
        res = requests.get(api_url, headers=headers)
        res.encoding = 'utf-8'
        #debugging
        print('flipped name')
        print(res)
        #########
        if res.status_code == 404:
            search_url = 'https://api.artsy.net/api/search?q='+ name
            res = requests.get(search_url, headers=headers)
            res.encoding = 'utf-8'
            try:
                t = res.json()['_embedded']['results'][0]['type']
            except:
                t = []
            if t  != 'artist':
                return {'name' : " ".join(artist_name), 'skip':True}  
            api_url = res.json()['_embedded']['results'][0]['_links']['self']['href']
            res = requests.get(api_url, headers=headers)
            res.encoding = 'utf-8'
            #debugging
            print('search query')
            print(res)
            ##########
            return res.json()
        return res.json()
    

In [41]:
search_url = 'https://api.artsy.net/api/search?q='+ '-'.join(['Serebrjakowa', 'Sinaida', 'Jewgenewna'])
res = requests.get(search_url, headers=headers)
res.encoding = 'utf-8'

In [43]:
res.json()['_embedded']

{'results': []}

In [45]:
query_json(['Serebrjakowa', 'Sinaida', 'Jewgenewna'])

flipped name
<Response [404]>


{'name': 'Serebrjakowa Sinaida Jewgenewna', 'skip': True}

In [15]:
def create_genes(artist_json):
    genes_toadd = []
    if artist_json.get('skip'):
        return []
    else:
        x = artist_json['_links']['genes']['href']
    genes = requests.get(x, headers=headers)
    gene_json = genes.json()['_embedded']['genes']
    artist_genes = []
    for gene in gene_json:
        if gene['display_name'] != None:
            artist_genes.append(gene['display_name']) 
        else: 
            artist_genes.append(gene['name'])
    gene_objects = []
    for gene in artist_genes:
        existing = session.query(Gene).filter(Gene.name == gene).first()
        if existing:
            gene_objects.append(existing)
        else:
            gene_objects.append(Gene(name=gene))
    return gene_objects

In [16]:
session.query(Artist).filter(Artist.name == None).first()

In [17]:
def create_artist(artist_name):
    artist_json = query_json(artist_name)
    existing = session.query(Artist).filter(Artist.name == artist_json['name']).first()
    if existing:
        return existing
    else:
        n = artist_json.get('name')
        g = artist_json.get('gender')
        bday = artist_json.get('birthday')
        dday = artist_json.get('deathday')
        loc = artist_json.get('location')
        nat = artist_json.get('nationality')
        gen = create_genes(artist_json)
        return Artist(name=n, gender=g, birthday=bday,
                     deathday=dday, location=loc, nationality=nat, genes=gen)

In [25]:
def create_database_year(year):
    for row in year:
        print(row['Rank'])
        name = row['Name']
        artist_object = create_artist(name)
        time.sleep(1.5)
        year = Year(year=row['SaleYear'],
                    rank=row['Rank'],
                    artist=artist_object,
                    totalsold=row['TotalSold'],
                    totallots=row['TotalLots'],
                    maxprice=row['MaxPrice'])
        session.add(year)
        session.commit()

In [33]:
from artists2008091415 import clean2008top500, clean2009top500, clean2014top500, clean2015top500

In [95]:
create_database_year(clean2008top500)

1
verified true
<Response [200]>
2
verified true
<Response [200]>
3
verified true
<Response [200]>
4
verified true
<Response [200]>
5
verified true
<Response [200]>
6
verified true
<Response [200]>
7
verified true
<Response [200]>
8
verified true
<Response [200]>
9
verified true
<Response [200]>
10
verified true
<Response [200]>
11
verified true
<Response [200]>
12
verified true
<Response [200]>
13
verified true
<Response [200]>
14
verified true
<Response [200]>
15
verified true
<Response [200]>
16
verified true
<Response [200]>
17
verified true
<Response [200]>
18
verified true
<Response [200]>
19
verified true
<Response [200]>
20
flipped name
<Response [404]>
search query
<Response [200]>
21
verified true
<Response [200]>
22
flipped name
<Response [404]>
search query
<Response [200]>
23
verified true
<Response [200]>
24
verified true
<Response [200]>
25
verified true
<Response [200]>
26
verified true
<Response [200]>
27
verified true
<Response [200]>
28
verified true
<Response [200]>

198
verified true
<Response [200]>
199
verified true
<Response [200]>
200
flipped name
<Response [404]>
search query
<Response [200]>
201
flipped name
<Response [404]>
search query
<Response [200]>
202
verified true
<Response [200]>
203
verified true
<Response [200]>
204
flipped name
<Response [404]>
search query
<Response [200]>
205
verified true
<Response [200]>
206
verified true
<Response [200]>
207
verified true
<Response [200]>
208
flipped name
<Response [200]>
209
verified true
<Response [200]>
210
flipped name
<Response [404]>
search query
<Response [200]>
211
flipped name
<Response [200]>
212
verified true
<Response [200]>
213
flipped name
<Response [200]>
214
flipped name
<Response [200]>
215
verified true
<Response [200]>
216
verified true
<Response [200]>
217
flipped name
<Response [404]>
search query
<Response [200]>
218
verified true
<Response [200]>
219
flipped name
<Response [404]>
220
verified true
<Response [200]>
221
flipped name
<Response [200]>
222
verified true
<Re

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [89]:
x = session.query(Gene).filter(Gene.name == '"Bad Painting"').all()

In [90]:
listy = []
for y in x:
    listy.append(y.name)

In [94]:
x[0].artists[0].name

'Neil Jenney'